# Updating IODP site data (Excel files for KML conversion)

Proposed sites, scheduled expeditions, and recently completed expeditions from https://www.iodp.org/resources/maps-and-kml-tools

---

There are 3 KML files on the IODP Maps & Resources webpage that need to be updated a few times a year:
- Proposed sites
- Scheduled expeditions
- Recently completed expeditions

These files are for IODP holes that haven't yet made it into the Drilled Holes KML, which contains comprehensive drilling data from the ship operator. Before our office gets data from the ship operators, we can track the status of the holes throughout the proposal and expedition stages.

In [1]:
# import libraries
import requests
from bs4 import BeautifulSoup
import os
import re
import sys
import urllib
import numpy as np
import pandas as pd

In [2]:
# URLs with information to update Excel files
url1 = "https://www.iodp.org/proposals/active-proposals"
url2 = "https://ssdb.iodp.org/SSDBupload/site_information.php?propID=ALL"
url3 = "https://www.iodp.org/expeditions/expeditions-schedule"

## Proposed Sites Excel file

ProposedSites_MonthYear.xlsx contains the following data:  
**Site Name / Name / Latitude / Longitude / Lead Proponent / Stage / Platform / Icon / IconScale / LabelScale / AppendData / IconColor / Description**

The data from the old file needs to be updated (add new proposed sites, remove de-activated or scheduled proposed sites).

First, we will add rows to the Excel file by pulling information from the [Active Proposals page](https://www.iodp.org/proposals/active-proposals) and [SIF site list](https://ssdb.iodp.org/SSDBupload/site_information.php?propID=ALL).  
From the Active Proposals page, we need the Proposal Number, Short Title, Lead Proponent, Platform, and Stage.  
From the SIF site list, we need the Site Name, Proposal ID, Latitude, and Longitude.

In [3]:
# read in existing Excel file to find last proposal number and create new Excel file
old_excel_file = 'ProposedSites_May2018.xlsx'
new_excel_file = 'ProposedSites_September2018.xlsx'

# copy contents of old Excel file to new one for updating
import shutil
shutil.copy(old_excel_file, new_excel_file)

# get last proposal number from old Excel file
df = pd.read_excel(new_excel_file)
lastprop = df['Name'].iloc[-1]

# read Active Proposals page html into string
try:
    page = urllib.request.urlopen(url1).read()
except:
    print("Cannot read url")

# read into beautifulsoup for parsing
soup = BeautifulSoup(page,'html.parser')
#proposals = soup.tbody

# get information for new proposals to append to Excel file
table = soup.find("table", attrs={"class":"tablesorter"})
headings = [th.get_text() for th in table.find("tr").find_all("th")]
table_body = table.find('tbody')

data = []
rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])
#print(data)

# create pandas dataframe and extract relevant fields
propdf = pd.DataFrame(data, columns=headings)
del propdf['Type']
del propdf['Ocean Basin']

# save copy of dataframe for later use
propdfsave = propdf.copy()

# remove all rows above last proposal number already in Excel file
lastind = propdf[propdf['Proposal Number'] == str(lastprop)].index[0]
propdf = propdf.iloc[lastind+1::]

# add P to beginning of proposal number column to match SIF list
propdf['Proposal Number'] = 'P'+propdf['Proposal Number']

# get list of proposal numbers to add to Excel file
proplist = propdf['Proposal Number'].tolist()

propdf

,Proposal Number,Short Title,Lead Proponent,Platform,Stage
84,P932,Hellenic Arc Volcanic Field,Druitt,JOIDES Resolution,SEP
85,P933,NW African Continental Margin Climate,Bickert,JOIDES Resolution,SEP
86,P934,Arctic Atlantic Gateway Climate,Geissler,JOIDES Resolution,SEP
87,P935,Arctic Fluid Flow Systems,Bünz,JOIDES Resolution,SEP
88,P937,Deepening Hole U1309D,McCaig,JOIDES Resolution,SEP
89,P939,Tohoku Petit-Spot Magmatism,Yamaguchi,Chikyu,SEP


These are the new proposals with sites that need to be added to the Excel file.

Next, we'll get the sites associated with each proposal.

In [4]:
# read SIF site list html into string
try:
    page = urllib.request.urlopen(url2).read()
except:
    print("Cannot read url")

# read into beautifulsoup for parsing
soup = BeautifulSoup(page,'html.parser')
table_body = soup.table

data = []
rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])
#print(data)

# create pandas dataframe for SIF site list
sitedf = pd.DataFrame(data)
sitedf.columns = ['Site Name','Proposal Number','Latitude','Longitude']

# search through dataframe for rows that match proposal list
#for ii in range(len(proplist)):
#    print(sitedf[sitedf[1] == proplist[ii]], propdf[propdf['Proposal Number'] == proplist[ii]])

# merge proposal list and site list
newdf = propdf.merge(sitedf, on='Proposal Number')

# add other columns and reorganize to match Excel file
newdf['Icon']='208'
newdf['IconScale']='3'
newdf['LabelScale']='3'
newdf['AppendData']='Yes'
newdf['IconColor']='fuchsia'

newdf = newdf.rename(index=str, columns={"Proposal Number":"Name", "Short Title":"Description"})

newdf = newdf[['Site Name', 'Name', 'Latitude', 'Longitude', 'Lead Proponent', 'Stage', \
               'Platform', 'Icon', 'IconScale', 'LabelScale', 'AppendData', 'IconColor', 'Description']]

# rename fields within a column to be consistent with previous content
newdf.loc[newdf.Platform == 'JOIDES Resolution', 'Platform'] = "JR"
newdf.loc[newdf.Platform == 'Mission Specific Platform', 'Platform'] = "MSP"

# remove 'P' from Proposal Name to be consistent with previous content
newdf['Name'] = newdf['Name'].str[1:]

newdf.head()

,Site Name,Name,Latitude,Longitude,Lead Proponent,Stage,Platform,Icon,IconScale,LabelScale,AppendData,IconColor,Description
0,CSK-01A,932,36.7293,25.6482,Druitt,SEP,JR,208,3,3,Yes,fuchsia,Hellenic Arc Volcanic Field
1,CSK-02A,932,36.7438,25.7146,Druitt,SEP,JR,208,3,3,Yes,fuchsia,Hellenic Arc Volcanic Field
2,CSK-03A,932,36.5549,25.4398,Druitt,SEP,JR,208,3,3,Yes,fuchsia,Hellenic Arc Volcanic Field
3,CSK-04A,932,36.5728,25.4092,Druitt,SEP,JR,208,3,3,Yes,fuchsia,Hellenic Arc Volcanic Field
4,CSK-05A,932,36.4355,25.3805,Druitt,SEP,JR,208,3,3,Yes,fuchsia,Hellenic Arc Volcanic Field


These are all the sites for the new proposals that will be added to the Excel file.

Next, we can update and save our results to the Excel file.

Helpful XlsxWriter documentation can be found here: https://xlsxwriter.readthedocs.io/working_with_pandas.html

In [5]:
# export dataframe and append to Excel file
writer = pd.ExcelWriter(new_excel_file, engine='xlsxwriter')
df.to_excel(writer, index=False)
newdf.to_excel(writer, startrow=len(df)+1, index=False, header=False)

# get xlsxwriter objects from the dataframe writer object
#workbook = writer.book
#worksheet = writer.sheets['Sheet1']

# add some cell formats
#cell_format = workbook.add_format()
#cell_format.set_align('left')
#worksheet.set_column('A:D', 10)

writer.save()

Now the Proposed Sites Excel file has been updated with the new proposed sites.  

Next, we'll remove the sites for proposals that have been deactivated.  

**Note: This part will have to be updated manually, as the number of de-activated proposals will vary each time we run the code. A list of de-activated proposals will need to be obtained after each SEP meeting.**

In [6]:
# read in edited Excel file
df = pd.read_excel(new_excel_file)

# make sure values in 'Name' column are strings
df['Name'] = df['Name'].astype(str)

# get index values corresponding to proposals that have been de-activated
# !!! must update proposal IDs of de-activated proposals here !!!
idx = df.index[df['Name'] == '914'].tolist()
idx.extend(df.index[df['Name'] == '922'].tolist())
idx.extend(df.index[df['Name'] == '930'].tolist())

# drop rows from dataframe
newdf = df.drop(df.index[idx])

# replace dataframe in Excel file
writer = pd.ExcelWriter(new_excel_file, engine='xlsxwriter')
newdf.to_excel(writer, index=False)
writer.save()

If the proposal has been scheduled as an expedition, we'll remove the relevant sites from the Proposed Sites Excel file and put them into the Scheduled Expeditions Excel file. We can list out the Expeditions here as well, so we know the sites associated with the proposals should show up in the other files.

In [7]:
# read in updated proposed sites Excel file
df = pd.read_excel(new_excel_file)

# search Active Proposals webpage for 'Exp' in Stage column and get corresponding proposal number
expdf = propdfsave[propdfsave['Stage'].str.match("Exp")]
expdf = expdf.rename(index=str, columns={"Proposal Number":"Name"})

# find the proposal-expeditions that show up in the Excel file (if there are any)
deldf = pd.merge(expdf, df, on='Name')

# remove proposal-expeditions from Excel file
dellist = deldf.Name.unique().tolist()

idx = []
for i in dellist:
    idx.extend(df.index[df['Name'] == i].tolist())

# drop rows from dataframe
newdf = df.drop(df.index[idx])

# replace dataframe in Excel file
writer = pd.ExcelWriter(new_excel_file, engine='xlsxwriter')
newdf.to_excel(writer, index=False)
writer.save()

# output list of proposals that have been scheduled as expeditions
# because they may need to go into Scheduled Expeditions Excel file
print('These proposals have been scheduled as expeditions: ',deldf.Name.unique())
print('Make sure they are in the Scheduled Expeditions Excel file.')

These proposals have been scheduled as expeditions:  ['716' '834' '846' '853' '859' '864' '890']
Make sure they are in the Scheduled Expeditions Excel file.


## Scheduled Expeditions Excel file

The top table on the [Expeditions Schedule page](https://www.iodp.org/expeditions/expeditions-schedule) shows proposals that have been scheduled as expeditions. The proposed drilling sites for these expeditions should be listed in the Scheduled Expeditions Excel file.

ScheduledExpeditions_MonthYear.xlsx contains the following data:  
**Site Name / Latitude / Longitude / Name / Schedule / Icon / IconColor / IconScale / LabelScale / AppendDataColumnsToDescription / Description / Original Proposal / Drilling Platform / Co-chief Scientists / web info **

We'll need to find out which expeditions should be removed from the Scheduled Expeditions Excel file (ones that are no longer in the table on the webpage) and which expeditions should be added to the Scheduled Expeditions Excel file (ones that have progressed from Proposed Sites to Scheduled Expeditions).

In [8]:
# get expeditions listed in the Excel file
old_excel_file = 'ScheduledExpeditions_May2018.xlsx'
new_excel_file = 'ScheduledExpeditions_September2018.xlsx'

# copy contents of old Excel file to new one for updating
shutil.copy(old_excel_file, new_excel_file)

# read in old Excel file
df = pd.read_excel(old_excel_file)

# remove 'Exp ' from Name column in Excel file
#df['Name'] = df['Name'].str[4:]

# get list of expeditions in the Excel file
xlexplist = df['Name'].str[4:].unique().tolist()

# read Expeditions Schedule page html into string
try:
    page = urllib.request.urlopen(url3).read()
except:
    print("Cannot read url")

# read into beautifulsoup for parsing
soup = BeautifulSoup(page,'html.parser')
#proposals = soup.tbody

# get information for scheduled expeditions
table = soup.find("table", attrs={"id":"scheduledExpeditionsTable"})
headings = [th.get_text() for th in table.find("tr").find_all("th")]
table_body = table.find('tbody')

data = []
rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])
#print(data)

# create pandas dataframe
expdf = pd.DataFrame(data, columns=headings).dropna()
expdf = expdf.rename(index=str, columns={"#":"Name"})

# get list of scheduled expeditions from the webpage
siteexplist = expdf.Name.tolist()

# get difference between lists of expeditions on the webpage and in the Excel file

# expeditions that are in the Excel file but not on the webpage (remove from Excel)
rmfromxl = list(set(xlexplist) - set(siteexplist))

# expeditions that are on the webpage but not in the Excel file (add to Excel)
addtoxl = list(set(siteexplist) - set(xlexplist))

print('These expeditions should be removed from the Excel file: ',rmfromxl)
print('Make sure they are in the Recently Completed Expeditions Excel file.')
print('')
print('These expeditions should be added to the Excel file: ',addtoxl)
#print('The expeditions should correspond to the following proposals: ',deldf.Name.unique())

#expdf

These expeditions should be removed from the Excel file:  ['376']
Make sure they are in the Recently Completed Expeditions Excel file.

These expeditions should be added to the Excel file:  ['387', '390', '391', '388', '389', '392', '393']


Now that we know which expeditions and respective sites should be included in the Excel file, we need to get the data formatted correctly.

In [10]:
# add the proposals that have been scheduled as expeditions (removed from Proposed Sites Excel file)
# to the Scheduled Expeditions Excel file

# table of proposals and corresponding expeditions that were recently scheduled (from Proposed Sites Excel file)
prop2expdf = deldf[['Name','Stage_x']].drop_duplicates()
prop2expdf = prop2expdf.rename(index=str, columns={"Name":"Proposal Number", "Stage_x":"Expedition"})
prop2expdf['Expedition'] = prop2expdf['Expedition'].str[3:]

# note: the expeditions listed in prop2expdf may not match the above lists if multiple expeditions are scheduled
# from one proposal (e.g., Proposal 853 and Expeditions 390/393)

# we will use the list of expeditions in 'addtoxl' and find the corresponding proposed sites to add to the
# Scheduled Expeditions Excel file

# add blank columns to relevant proposed sites dataframe
deldf["Schedule"] = np.nan
deldf["web info"] = np.nan

# copy relevant columns from proposed sites into new dataframe
newdf = deldf[['Site Name','Latitude','Longitude','Stage_x','Schedule','Icon','IconColor','IconScale','LabelScale',\
               'AppendDataColumnsToDescription','Description','Name','Platform_x','Lead Proponent_x','web info']]

# rename headers to match Scheduled Expeditions Excel file
newdf = newdf.rename(index=str, columns={"Name":"Original Proposal", "Platform_x":"Drilling Platform", \
                                         "Stage_x":"Name", "Lead Proponent_x":"Co-chief Scientists"})

# update miscellaneous column data and formatting
newdf.loc[newdf.IconColor == 'fuchsia', 'IconColor'] = "cyan"
newdf['Name'] = 'Exp '+newdf['Name'].str[3:]
df['Name'] ='Exp '+df['Name']

#newdf

# export dataframe and append to Scheduled Expeditions Excel file
writer = pd.ExcelWriter(new_excel_file, engine='xlsxwriter')
df.to_excel(writer, index=False)
newdf.to_excel(writer, startrow=len(df)+1, index=False, header=False)

writer.save()

# note: check Expedition name column ... make sure 'Exp Exp ###' doesn't show up

Now we have the table that will be appended to the Scheduled Expeditions Excel file.

The 'Schedule', 'Original Proposal', 'Co-chief Scientists', and 'web info' columns will need to be updated or filled in manually. Go to the [Expeditions Schedule page](https://www.iodp.org/expeditions/expeditions-schedule) and use the dates and links to JRSO's expedition sites to find the appropriate information to enter in the Excel file.

Remember to manually cut and paste the relevant rows of any recently completed expeditions mentioned above from the Scheduled Expeditions to the Recently Completed Expeditions Excel file.

In [21]:
# remove expeditions from Scheduled Expeditions Excel file (to be moved to Recently Completed Expeditions)

# prepare files for recently completed expeditions
old_excel_file = 'RecentlyCompletedExpeditions_May2018.xlsx'
new_excel_file = 'RecentlyCompletedExpeditions_September2018.xlsx'

# copy contents of old Excel file to new one for updating
shutil.copy(old_excel_file, new_excel_file)

print('These expeditions should be removed from the Scheduled Expeditions Excel file: ',rmfromxl)

These expeditions should be removed from the Scheduled Expeditions Excel file:  ['376']


## Recently Completed Expeditions Excel file

The bottom table on the [Expeditions Schedule page](https://www.iodp.org/expeditions/expeditions-schedule) shows expeditions that have been scheduled and completed. The actual drilling sites for these expeditions should be listed in their respective Scientific Prospectus documents.

RecentlyCompletedExpeditions_MonthYear.xlsx contains the following data:  
**Site Name / Latitude / Longitude / Name / Schedule / Icon / IconColor / IconScale / LabelScale / AppendDataColumnsToDescription / Description / Original Proposal / Drilling Platform / Co-chief Scientists / web info **

Recently completed expeditions may not have been added to the Drilled Holes KML file on [iodp.org](https://www.iodp.org/resources/maps-and-kml-tools). If this is the case (i.e., JRSO, ESO, and CDEX have not responded to our request to update the drilled holes database), we need to add the data to the Recently Completed Expeditions Excel file.

In [23]:
# move expeditions that have been scheduled and completed to the Recently Completed Expeditions file

# this was done manually as the last part of the previous section

Open the Drilled Holes KML file to check the most recent expeditions included. Remove those expeditions from the Excel file.

Check the Excel files carefully before running the KML conversion code.

Now you're done!!!